In [ ]:
import requests
import json
import time
import pandas as pd
from googletrans import Translator
import config

# Atualizar credenciais

In [ ]:
payload = ""
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer ' + config.bearer,
  'Cookie':  config.cookie
}

hostname = config.url

In [ ]:
def get_categories(hostname, init, headers, payload):
    
    categories_id=[init]
    
    for i in categories_id:
        url = f"https://{hostname}/v3/categories/{i}"
        
        response = requests.request("GET", url, headers=headers, data=payload)
        # print(response.text)
        jsondata = response.json()
        
        new_categories = [d['entity']['target_id'] for d in jsondata['entity']['has_types_category_rels']]
        categories_id.extend(new_categories)
        
    return(categories_id)


def get_artifacts(hostname, artifact_type, headers, payload, pages):

    list = []
    
    for i in range(pages):
        url = f"https://{hostname}/v3/governance_artifact_types/{artifact_type}?workflow_status=published&limit=200&offset={i*200}"
        
        response = requests.request("GET", url, headers=headers, data=payload)
        jsondata = response.json()
        
        list.extend(jsondata['resources'])
        
        print(artifact_type + ' página: '+ f'{i}')

    df = pd.DataFrame(list)
    df['parent_category'] = [d['artifact_id'] for d in df['parent_category']]
    
    return(df)


def google_translator(input):
    #translating using Google Translator API
    translator = Translator()
    translated = translator.translate(input, dest = 'pt', src = 'en')
    return(translated.text)


def tradutor(hostname, payload, headers, df, artifact_type, version, location):
    
    contador = 1
    
    for i in df.index:
        
        artifact_id = df.loc[i]['artifact_id']
        version_id = df.loc[i]['version_id']

        #getting long_description
        print('getting long_description')
        url = f"https://{hostname}/{version}/{artifact_type}/{artifact_id}/versions"
        response = requests.request("GET", url, headers=headers, data=payload)
        # print(response.text)
        jsondata = response.json()
        long_description = jsondata[location][0]['entity']['long_description']
        name = jsondata[location][0]['metadata']['name']
        
        print(name)
        print(long_description)
        
    
        translated_long_description = google_translator(long_description)
        translated_name = google_translator(name)

        #updating the new long_description
        print('updating the new long_description')
        url = f"https://{hostname}/{version}/{artifact_type}/{artifact_id}/versions/{version_id}"
        
        payload = json.dumps({
            "long_description": translated_long_description,
            "name": translated_name,
            "revision": "1"
        })
        
        response = requests.request("PATCH", url, headers=headers, data=payload)
        jsondata = response.json()
        print(jsondata)
        
        print('%%%%%%% Completed: ' + f'{contador}' + '/' + f'{len(df.index)}')
        
        contador = contador + 1


In [ ]:
cross = '7b443c89-5306-4069-90b4-af58479d800e' #category ID

categories_id_cross = get_categories(hostname, cross, headers, payload)
print(len(categories_id_cross))

In [ ]:
# artifact_type = ['classification','classifications'] 
# artifact_type = ['data_class','data_classes'] 
# artifact_type = ['reference_data','reference_data_sets'] 
# artifact_type = ['policy','policies']
# artifact_type = ['rule','rules'] 
artifact_type = ['glossary_term','glossary_terms']

df = get_artifacts(hostname, artifact_type[0], headers, payload, 180)
df

In [ ]:
df_cross = df[df['parent_category'].isin(categories_id_cross)]
print(len(df_cross))

In [ ]:
# classification, data_class, policy, rule
version='v3'
location='resources'

tradutor(hostname, payload, headers, df_accelerator, artifact_type[1], version, location)

In [ ]:
## reference_data
version='v4'
location='versions'

tradutor(hostname, payload, headers, df_cross, artifact_type[1], version, location)

In [ ]:
## business_terms
version='v3'
location='resources'

tradutor(hostname, payload, headers, df_cross, artifact_type[1], version, location)

In [ ]:
#TRANSLATING CATEGORY NAMES

for categoria in [categories_id_cross]:
    
    print('#####################################')
    print(categoria)
    print('#####################################')
    
    contador = 1
    
    for i in range(len(categoria)):
        category_id = categoria[i]
    
        url = f"https://{hostname}/v3/categories/{category_id}"
        
        response = requests.request("GET", url, headers=headers, data=payload)
        jsondata = response.json()
    
    
        long_description = jsondata['entity']['long_description']
        name = jsondata['metadata']['name']
    
        print(name)
        print(long_description)
        
        translated_long_description = google_translator(long_description)
        translated_name = google_translator(name)
    
        print(translated_name)
        print(translated_long_description)
    
        #updating the new long_description
        print('updating the new long_description')
        url = f"https://{hostname}/v3/categories/{category_id}"
        
        payload = json.dumps({
            "long_description": translated_long_description,
            "name": translated_name
        })
        
        response = requests.request("PATCH", url, headers=headers, data=payload)
        jsondata = response.json()
        print(jsondata)
        
        print('%%%%%%% Completed: ' + f'{contador}' + '/' + f'{len(categoria)}')
        
        contador = contador + 1